# Uebung 4 Mehrklassenklassifikation mit Neuronalen Netzen


## Imports

In [1]:
import os 
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator 


/opt/anaconda3/envs/BGA2_macos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/BGA2_macos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/BGA2_macos/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/BGA2_macos/lib/python3.6/site-packages

## Globale Variablen
Definiere an dieser Stelle alle Variablen, die global verwendet werden, z.B.: Pfadnamen

In [2]:
# training images
DATA_PATH_FINAL_TRAINING_IMAGES = './dataset/GTSRB/Final_Training/Images'
assert os.path.exists(DATA_PATH_FINAL_TRAINING_IMAGES), "Der angegebene Pfad existriert nicht."

In [3]:
# croped images
DATA_PATH_FINAL_TRAINING_IMAGES_CROP = './dataset/GTSRB_img_Crop/Final_Training/Images'
assert os.path.exists(DATA_PATH_FINAL_TRAINING_IMAGES), "Der angegebene Pfad existriert nicht."

## Datenaufbereitung
Hinweise findest du hier: https://keras.io/getting_started/intro_to_keras_for_engineers/#data-loading-amp-preprocessing

In [4]:
path = DATA_PATH_FINAL_TRAINING_IMAGES

'''
在GTSRB文件夹所在的位置生成一个修剪过的GTSRB_img_Crop文件夹，里面包含根据csv文件里的roi修剪的图像

'''

shutil.rmtree('./dataset/GTSRB_img_Crop/') # 删除文件夹，请确保路径正确


# 从每个标志的文件夹中找到csv文件
csv_files = []
for dirpath, dirnames, filenames in os.walk(path, topdown=False):
    for filename in filenames:
        if filename.endswith('.csv'):
            csv_files.append(os.path.join(dirpath, filename))



class TrafficSign:
    trafficSign_name = ''
    left_top_x = 0,
    left_top_y = 0,
    right_bottom_x = 0,
    right_bottom_y = 0,
    width = 0,
    height = 0,
    label = ''

    def tostring(self):
        print([self.trafficSign_name,
               self.width, self.height,
               self.left_top_x, self.left_top_y,
               self.right_bottom_x, self.right_bottom_y,
               self.label])


for csv in csv_files:
    base_path = os.path.dirname(csv)
    # read csv data
    trafficSigns = []
    with open(csv) as file:
        for line in file:
            if line.find('.ppm') == -1:
                continue
            raw_data = line.split(';')
            trafficSign = TrafficSign()
            trafficSign.trafficSign_name = raw_data[0]
            trafficSign.width = int(raw_data[1])
            trafficSign.height = int(raw_data[2])
            trafficSign.left_top_x = int(raw_data[3])
            trafficSign.left_top_y = int(raw_data[4])
            trafficSign.right_bottom_x = int(raw_data[5])
            trafficSign.right_bottom_y = int(raw_data[6])
            trafficSign.label = raw_data[7]
            trafficSigns.append(trafficSign)

    # crop each image according to the csv in this folder
    for dirpath, dirnames, filenames in os.walk(base_path, topdown=False):
        for filename in filenames:
            if not filename.endswith('.ppm'):
                continue
            fullPath = os.path.join(dirpath, filename)
            for sign in trafficSigns:
                if filename == sign.trafficSign_name:
                    image = Image.open(fullPath)
                    # start cropping according to this sign
                    region = (sign.left_top_x, sign.left_top_y, sign.right_bottom_x, sign.right_bottom_y)
                    image_crop = image.crop(region)
                    # update the new image path
                    newFullPath = fullPath.replace('GTSRB', 'GTSRB_img_Crop')
                    newFullPath = newFullPath.replace('.ppm', '.jpg')
                    if not os.path.exists(os.path.dirname(newFullPath)):
                        os.makedirs(os.path.dirname(newFullPath))
                    # save the images
                    image_crop.save(newFullPath)
                    break


In [5]:
''' 把训练集划分成训练集和验证集 '''

path = DATA_PATH_FINAL_TRAINING_IMAGES_CROP

dirs = []
split_percentage = 0.2

for dirpath, dirnames, filenames in os.walk(path, topdown=False):
   for dirname in dirnames:
       fullpath = os.path.join(dirpath, dirname)
       fileCount = len([name for name in os.listdir(fullpath) if os.path.isfile(os.path.join(fullpath, name))])
       files = os.listdir(fullpath)
       for index in range((int)(split_percentage * fileCount)):
           newIndex = random.randint(0, fileCount - 1)
           fullFilePath = os.path.join(fullpath, files[newIndex])
           newFullFilePath = fullFilePath.replace('Final_Training', 'Final_Validation')
           base_new_path = os.path.dirname(newFullFilePath)
           if not os.path.exists(base_new_path):
               os.makedirs(base_new_path)
           # move the file
           try:
               shutil.move(fullFilePath, newFullFilePath)
           except IOError as error:
               print('skip moving from %s => %s' % (fullFilePath, newFullFilePath))


skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00009_00000.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00009_00000.jpg
skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00007_00029.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00007_00029.jpg
skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00004_00018.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00004_00018.jpg
skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00005_00021.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00005_00021.jpg
skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00018_00020.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00018_00020.jpg
skip moving from ./dataset/GTSRB_img_Crop/Final_Training/Images/00031/00007_00020.jpg => ./dataset/GTSRB_img_Crop/Final_Validation/Images/00031/00007_00020.jpg
skip moving from ./dataset/GTSRB_img_Cro

In [6]:
# path of training set and validation set
train_set_base_dir = './dataset/GTSRB_img_Crop/Final_Training/Images'
validation_set_base_dir = './dataset/GTSRB_img_Crop/Final_Validation/Images'

# create objects of data generator
# 如果出现内存溢出的问题需要使用生成器训练
train_datagen = ImageDataGenerator(rescale=1. / 255) #把图像的值压缩为0-1之间

'''
图片生成器可以对图片进行随机的位移/缩放/旋转，所谓数据提升(数据增强)

其他参数：

rotation_range：是一个0~180的度数，用来指定随机选择图片的角度。
width_shift和height_shift：用来指定水平和竖直方向随机移动的程度，这是两个0~1之间的比例。
rescale：值将在执行其他处理前乘到整个图像上，我们的图像在RGB通道都是0~255的整数，这样的操作可能使图像的值过高或过低，所以我们将这个值定为0~1之间的数。
shear_range：是用来进行剪切变换的程度，参考剪切变换
zoom_range：用来进行随机的放大
horizontal_flip：随机的对图片进行水平翻转，这个参数适用于水平翻转不影响图片语义的时候
fill_mode：用来指定当需要进行像素填充，如旋转，水平和竖直位移时，如何填充新出现的像素
'''

train_data_generator = train_datagen.flow_from_directory(
    directory=train_set_base_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical')
'''
以文件夹路径为参数,生成经过数据提升/归一化后的数据,在一个无限循环中无限产生batch数据

参数：
directory: 目标文件夹路径,对于每一个类,该文件夹都要包含一个子文件夹.子文件夹中任何JPG、PNG、BNP、PPM的图片都会被生成器使用.

target_size: 整数tuple,默认为(256, 256). 图像将被resize成该尺寸

color_mode: 颜色模式,为"grayscale","rgb"之一,默认为"rgb".代表这些图片是否会被转换为单通道或三通道的图片.

classes: 可选参数,为子文件夹的列表,如['dogs','cats']默认为None. 若未提供,则该类别列表将从directory下的子文件夹名称/结构自动推断。每一个子文件夹都会被认为是一个新的类。
(类别的顺序将按照字母表顺序映射到标签值)。通过属性class_indices可获得文件夹名与类的序号的对应字典。

class_mode: "categorical", "binary", "sparse"或None之一. 默认为"categorical. 
该参数决定了返回的标签数组的形式, "categorical"会返回2D的one-hot编码标签,"binary"返回1D的二值标签."sparse"返回1D的整数标签,如果为None则不返回任何标签, 生成器将仅仅生成batch数据, 这种情况在使用model.predict_generator()和model.evaluate_generator()等函数时会用到.

batch_size: batch数据的大小,默认32

shuffle: 是否打乱数据,默认为True

seed: 可选参数,打乱数据和进行变换时的随机数种子

save_to_dir: None或字符串，该参数能让你将提升后的图片保存起来，用以可视化

save_prefix: 字符串，保存提升后图片时使用的前缀, 仅当设置了save_to_dir时生效

save_format: "png"或"jpeg"之一，指定保存图片的数据格式,默认"jpeg"

flollow_links: 是否访问子文件夹中的软链接
'''

validation_datagen = ImageDataGenerator(rescale=1. /255)

validation_data_generator = validation_datagen.flow_from_directory(
    directory=validation_set_base_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

Found 32100 images belonging to 43 classes.
Found 7109 images belonging to 43 classes.


## Aufbau des Modells
Zum Aufbau deines Modells kannst du dich an die gezeigten Beispiele richten. Implementiere zuerst ein einfaches Modell, welches du je nach Performance erweitern kannst. 

Unten findest du die Auflistung der Schichten (Layers), die du für dein Modell miteinander kombinieren kannst. 


Überlege dir, welche Layers für die Klassifikationsaufgabe mit HOG-Features gut sind und welche Layer sich für die Klassifikationsaufgabe mit ppm-Dateien eignen.

In [ ]:
# model = Sequential()
# model.add(layers.Conv2D(xyz))
# model.add(layers.Activation(xyz))
# model.add(layers.Conv2D(xyz))
# model.add(layers.Activation(xyz))
# model.add(layers.MaxPooling2D(xyz))
# model.add(layers.Dropout(xyz))
# model.add(layers.Flatten())
# model.add(layers.Dense(xyz))
# .
# .
# .
# model.add(layers.Activation('softmax'))

In [7]:
model = Sequential()

# add Con2D layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))

# flatten
model.add(Flatten())

# dropOut layer
model.add(Dropout(0.2))

# add one simple layer for classification
model.add(Dense(units=512, activation='relu'))

# add output layer
model.add(Dense(units=43, activation='softmax'))

## Kompilieren des Modells
Eine detaillierte Beschreibung der [compile](https://keras.io/api/models/model_training_apis/#compile-method)-Methode findest du in Keras API Referenz.

In [8]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [9]:
# model info
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 128)         147584    
__________

## Training des Modells
Eine detaillierte Beschreibung der [fit](https://keras.io/api/models/model_training_apis/#fit-method)-Methode findest du in Keras API Referenz.

In [ ]:
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_data_generator,
    validation_steps=50)

Epoch 1/30
  2/100 [..............................] - ETA: 1:54 - loss: 3.7510 - acc: 0.0469

## Evaluation des Modells
Eine detaillierte Beschreibung der [evaluate](https://keras.io/api/models/model_training_apis/#evaluate-method)-Methode findest du in Keras API Referenz.

Nach der Anwendung der *evaluate*-Methode kannst du dir zusätzlich den ausfuehrlichen Klassifikationsbericht (*classification_report()*) sowie die Konfusionsmatrix (*confusion_matrix()*) anschauen.  

In [ ]:
# Evaluiere das trainierte Modell mit den Testdaten


In [ ]:
# plot the roc curve
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Speichern des trainierten Modells
Zum Speichern des trainierten Modells kann *save*-Methode
Weiterfuehrende Informationen zu dieser Methode unter folgendem [Link](https://keras.io/api/models/model_saving_apis/) zu finden. 

In [ ]:
# Speichere das trainierte Modell, um dessen Wiederverwendung zu ermoeglichen
model.save('traffic_signs.h5')

## Nutzen des trainierten Modells zum Vorhersagen von Verkehrszeichen-Klassen 
Zum Wiederverwenden des trainierten gespeicherten Modells kann die [load_model](https://keras.io/api/models/model_saving_apis/#loadmodel-function)-Funktionverwendet werden.

Eine detaillierte Beschreibung der [predict](https://keras.io/api/models/model_training_apis/#predict-method)-Methode findest du in Keras API Referenz.

In [ ]:
# Lade das trainierte Modell und teste die Erkennung der Verkehrszeichen-Klassen mit eigenen Beispielen
